This noteb book consists of pipeline for determining the percentage of population of each county which lies whithin each service areas.The initial file generetion for the service areas were done in ArcGIS Online and ArcGISPro. The resulting shapefiles were then processed using this notebook

**Import Libraries and Define Constants**

In [1]:
import geopandas as gpd
import pandas as pd
import glob
import os
import re

# Constants
CENSUS_BLOCK_SHP = "./shapefiles/population_block.shp"
COUNTIES_SHP = './shapefiles/mo_county.shp'
COVERAGE_AREA_DIR = './service_areas_clipped/'
OUTPUT_DIR = './coverage_analysis_results/'
SQM_TO_SQMILES = 2589988.11  # Square meters to square miles conversion factor

**Aggregate Population by County**

In [2]:
def aggregate_population(file_path):
    census_block = gpd.read_file(file_path)
    agg_pop = census_block.groupby('COUNTYFP20')['POP20'].sum().reset_index()
    agg_pop.rename(columns={'COUNTYFP20': 'countyfips', 'POP20': 'total_county_population'}, inplace=True)
    return agg_pop

**Prepare County Shapefile and Process Coverage Area**

In [3]:
def prepare_county_shapefile(file_path, population_data):
    counties = gpd.read_file(file_path).to_crs(epsg=26915)  # Project to UTM Zone 15N
    counties = counties.merge(population_data, on='countyfips', how='inner')
    counties['county_area'] = counties.geometry.area / SQM_TO_SQMILES  # Convert area to square miles
    return counties.to_crs(epsg=26915)  


**Process Coverage Areas and Save Results Dynamically**

In [4]:
def process_coverage_areas(directory, counties):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Checking directory: {directory}")
    
    shapefiles = glob.glob(os.path.join(directory, '*.shp'))
    print(f"Found {len(shapefiles)} shapefiles to process.")
    
    if not shapefiles:
        print("No shapefiles found. Check the directory path.")
        return
    
    for file_path in shapefiles:
        print(f"Processing file: {file_path}")
        try:
            census_blocks_coverage = gpd.read_file(file_path).to_crs(epsg=26915)  # Ensure CRS matches your data's location
            census_blocks_coverage = census_blocks_coverage.rename(columns={'COUNTYFP20': 'countyfips', 'POP20': 'block_population'})
            census_blocks_coverage['coverage_area'] = census_blocks_coverage.geometry.area / SQM_TO_SQMILES  # Convert area to square miles
            coverage_agg = census_blocks_coverage.groupby('countyfips').agg({'block_population': 'sum', 'coverage_area': 'sum'}).reset_index()
            
            # Perform rounding right after calculations
            coverage_agg = coverage_agg.round(2)
            
            coverage_agg['countyfips'] = coverage_agg['countyfips'].astype(str)
            counties['countyfips'] = counties['countyfips'].astype(str)
            
            final_data = counties.merge(coverage_agg, on='countyfips', how='left')
            final_data['block_population'].fillna(0, inplace=True)
            final_data['coverage_area'].fillna(0, inplace=True)
            final_data['percent_population_cov'] = (final_data['block_population'] / final_data['total_county_population']) * 100
            final_data['percent_area_cov'] = (final_data['coverage_area'] / final_data['county_area']) * 100
            
            # Round final_data DataFrame as well
            final_data = final_data.round(2)
            
            base_name = os.path.basename(file_path)
            file_name_no_ext = os.path.splitext(base_name)[0]
            shapefile_output_path = os.path.join(OUTPUT_DIR, file_name_no_ext + '_coverage_analysis.shp')
            excel_output_path = os.path.join(OUTPUT_DIR, file_name_no_ext + '_coverage_analysis.xlsx')
            
            # Saving to Shapefile does not support rounding directly; handle data conversion/formatting in Excel and analysis
            final_data.to_file(shapefile_output_path)
            print(f"Saved coverage analysis to {shapefile_output_path}")
            
            # Prepare and save DataFrame without geometry column to Excel
            df_for_excel = final_data.drop(columns='geometry')
            df_for_excel.to_excel(excel_output_path, index=False)
            print(f"Saved coverage analysis to Excel format as {excel_output_path}")
        except Exception as e:
            print(f"Failed to process {file_path}: {e}")

# Execute the workflow
county_population = aggregate_population(CENSUS_BLOCK_SHP)
counties_prepared = prepare_county_shapefile(COUNTIES_SHP, county_population)
process_coverage_areas(COVERAGE_AREA_DIR, counties_prepared)

Checking directory: ./service_areas_clipped/
Found 4 shapefiles to process.
Processing file: ./service_areas_clipped\population_block_15mins.shp


C:\Users\alhassan\AppData\Local\Temp\ipykernel_15340\4114923060.py:41: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_data.to_file(shapefile_output_path)


Saved coverage analysis to ./coverage_analysis_results/population_block_15mins_coverage_analysis.shp
Saved coverage analysis to Excel format as ./coverage_analysis_results/population_block_15mins_coverage_analysis.xlsx
Processing file: ./service_areas_clipped\population_block_30_mins.shp


C:\Users\alhassan\AppData\Local\Temp\ipykernel_15340\4114923060.py:41: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_data.to_file(shapefile_output_path)


Saved coverage analysis to ./coverage_analysis_results/population_block_30_mins_coverage_analysis.shp
Saved coverage analysis to Excel format as ./coverage_analysis_results/population_block_30_mins_coverage_analysis.xlsx
Processing file: ./service_areas_clipped\population_block_45mins.shp


C:\Users\alhassan\AppData\Local\Temp\ipykernel_15340\4114923060.py:41: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_data.to_file(shapefile_output_path)


Saved coverage analysis to ./coverage_analysis_results/population_block_45mins_coverage_analysis.shp
Saved coverage analysis to Excel format as ./coverage_analysis_results/population_block_45mins_coverage_analysis.xlsx
Processing file: ./service_areas_clipped\population_block_60mins.shp


C:\Users\alhassan\AppData\Local\Temp\ipykernel_15340\4114923060.py:41: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_data.to_file(shapefile_output_path)


Saved coverage analysis to ./coverage_analysis_results/population_block_60mins_coverage_analysis.shp
Saved coverage analysis to Excel format as ./coverage_analysis_results/population_block_60mins_coverage_analysis.xlsx


In [5]:
def combine_excel_sheets(output_dir, combined_excel_name="combined_coverage_analysis.xlsx"):
    excel_files = glob.glob(os.path.join(output_dir, '*.xlsx'))
    
    # Initialize an empty DataFrame to store the combined data
    combined_df = pd.DataFrame()

    for excel_file in excel_files:
        # Read the current Excel file into a DataFrame
        df = pd.read_excel(excel_file)

        # If combined_df is empty, initialize it with the current df
        if combined_df.empty:
            combined_df = df
        else:
            # Merge using 'countyfips' while avoiding column duplication
            # Identify non-key columns that are present in both DataFrames
            common_cols = set(combined_df.columns) & set(df.columns) - {'countyfips'}
            # Rename these columns in df before merging to ensure uniqueness
            df = df.rename(columns={col: f"{col}_{os.path.basename(excel_file)}" for col in common_cols})
            # Perform the merge
            combined_df = pd.merge(combined_df, df, on="countyfips", how="outer")

    # Save the combined DataFrame to a new Excel file
    combined_excel_path = os.path.join(output_dir, combined_excel_name)
    combined_df.to_excel(combined_excel_path, index=False)
    print(f"Combined Excel file saved to {combined_excel_path}")

OUTPUT_DIR = './coverage_analysis_results/'  # Ensure this matches your output directory
combine_excel_sheets(OUTPUT_DIR)

Combined Excel file saved to ./coverage_analysis_results/combined_coverage_analysis.xlsx


The results from this an excel sheet which contains the percentage population for the various travel time areas